In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd

In [2]:
M = loadmat('../data/M.mat') ### dictionary

In [6]:
list(M.keys())

['__header__', '__version__', '__globals__', 'M']

In [12]:
Mij = M['M']

In [15]:
Mij[2,1]

12

In [16]:
Mij[2]

array([   0,   12, 6478, ...,    0,    0,    0], dtype=int32)

In [35]:
raw_info = pd.read_csv('../data/commuting_allinfo.csv',dtype=object,encoding='cp1252')

In [36]:
raw_info.head()

,State FIPS Code,County FIPS Code,State Name,County Name,Metropolitan Statistical Area FIPS Code,Metropolitan Statistical Area of County,State FIPS Code.1,County FIPS Code.1,State Name.1,County Name.1,Metropolitan Statistical Area FIPS Code.1,Metropolitan Statistical Area of County,Workers in Commuting Flow,Margin of Error
0,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,001,Alabama,Autauga County,33860,"Montgomery, AL","8,635",597
1,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,007,Alabama,Bibb County,13820,"Birmingham-Hoover, AL",16,23
2,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,013,Alabama,Butler County,NaN,NaN,4,7
3,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,021,Alabama,Chilton County,13820,"Birmingham-Hoover, AL",597,251
4,01,001,Alabama,Autauga County,33860,"Montgomery, AL",001,043,Alabama,Cullman County,NaN,NaN,27,42


In [38]:
raw_info.columns

Index(['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
       'Metropolitan Statistical Area FIPS Code',
       'Metropolitan Statistical Area  of County', 'State FIPS Code.1',
       'County FIPS Code.1', 'State Name.1', 'County Name.1',
       'Metropolitan Statistical Area FIPS Code.1',
       ' Metropolitan Statistical Area of County', 'Workers in Commuting Flow',
       'Margin of Error'],
      dtype='object')

In [41]:
columns = ['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
       'Metropolitan Statistical Area FIPS Code',
       'Metropolitan Statistical Area  of County', 'State FIPS Code.1',
       'County FIPS Code.1', 'State Name.1', 'County Name.1',
       'Metropolitan Statistical Area FIPS Code.1',
       ' Metropolitan Statistical Area of County', 'Workers in Commuting Flow',
       'Margin of Error']

In [84]:
for name in columns[:11]:
    print('name:', name)
    print('na: ', sum(raw_info[name].isna()))
    print('length of unique: ', len(set(raw_info[name])))
    print("_______________")

name: State FIPS Code
na:  1
length of unique:  55
_______________
name: County FIPS Code
na:  3
length of unique:  326
_______________
name: State Name
na:  3
length of unique:  53
_______________
name: County Name
na:  3
length of unique:  1956
_______________
name: Metropolitan Statistical Area FIPS Code
na:  57298
length of unique:  394
_______________
name: Metropolitan Statistical Area  of County
na:  57298
length of unique:  394
_______________
name: State FIPS Code.1
na:  1200
length of unique:  55
_______________
name: County FIPS Code.1
na:  1790
length of unique:  326
_______________
name: State Name.1
na:  3
length of unique:  56
_______________
name: County Name.1
na:  1790
length of unique:  1956
_______________
name: Metropolitan Statistical Area FIPS Code.1
na:  49509
length of unique:  394
_______________


In [86]:
## delete all the nan row
county_travel =  raw_info[['State FIPS Code', 'County FIPS Code', 'State Name', 'County Name',
        'State FIPS Code.1',
       'County FIPS Code.1', 'State Name.1', 'County Name.1',
       'Workers in Commuting Flow',
       'Margin of Error']]

In [93]:
county_travel_cleaned = county_travel.dropna()

In [99]:
data_types_dict = {'State FIPS Code':np.int32, 'County FIPS Code':np.int32, 
        'State FIPS Code.1':np.int32,
       'County FIPS Code.1':np.int32,
       'Workers in Commuting Flow':np.float64,
       'Margin of Error':np.float64}

In [104]:
county_travel_cleaned['State FIPS Code'][0]

'01'

In [105]:
df = pd.DataFrame()
for name in ['State FIPS Code', 'County FIPS Code', 
        'State FIPS Code.1',
       'County FIPS Code.1', 
       'Workers in Commuting Flow',
       'Margin of Error']:
    df[name] =  county_travel_cleaned[name].str.replace(',', '').astype(float) 

In [110]:
df['longfips'] = df['State FIPS Code']*1000+df['County FIPS Code']
df['longfips1'] = df['State FIPS Code.1']*1000+df['County FIPS Code.1']

In [111]:
df.head()

,State FIPS Code,County FIPS Code,State FIPS Code.1,County FIPS Code.1,Workers in Commuting Flow,Margin of Error,longfips,longfips1
0,1.0,1.0,1.0,1.0,8635.0,597.0,1001.0,1001.0
1,1.0,1.0,1.0,7.0,16.0,23.0,1001.0,1007.0
2,1.0,1.0,1.0,13.0,4.0,7.0,1001.0,1013.0
3,1.0,1.0,1.0,21.0,597.0,251.0,1001.0,1021.0
4,1.0,1.0,1.0,43.0,27.0,42.0,1001.0,1043.0


In [112]:
cn_all = list(df.longfips)+list(df.longfips1)

In [113]:
len(set(cn_all))
### it is larger the fips_number within US

3221

In [60]:
fips = loadmat('../data/countyfips_num.mat') ### dictionary

In [66]:
fips_num = fips['countyfips_num'].ravel()

In [120]:
difference = set(cn_all) - set(fips_num)

In [125]:
set(fips_num) - set(cn_all)
## new in the fips code 2158 (Kusilvak Census Area) previously as 
#  0227 (Alaska      Wade Hampton Census Area)
#^__________________________________________^#
## 46102 (Oglala Lakota County) previously as 
# 46113 (South Dakota      Shannon County)
#^__________________________________________^#
## 51515 Virginia   Bedford city is 
# Virginia, 2013: The independent city of Bedford (FIPS 51515) merges into
# Bedford County (FIPS 51019).

{2158, 46102}

In [133]:
df[df['longfips'] == 2270.0]

,State FIPS Code,County FIPS Code,State FIPS Code.1,County FIPS Code.1,Workers in Commuting Flow,Margin of Error,longfips,longfips1
3158,2.0,270.0,2.0,20.0,5.0,4.0,2270.0,2020.0
3159,2.0,270.0,2.0,50.0,7.0,6.0,2270.0,2050.0
3160,2.0,270.0,2.0,180.0,2.0,2.0,2270.0,2180.0
3161,2.0,270.0,2.0,185.0,2.0,3.0,2270.0,2185.0
3162,2.0,270.0,2.0,188.0,2.0,3.0,2270.0,2188.0
3163,2.0,270.0,2.0,270.0,1993.0,95.0,2270.0,2270.0
3164,2.0,270.0,2.0,290.0,1.0,3.0,2270.0,2290.0


In [122]:
for c_i in fips_num[:2]:
    df[df.longfips==c_i]

In [124]:
int(df[df.longfips==c_i]['longfips1']

42      1003.0
43      1005.0
44      1015.0
45      1023.0
46      1025.0
        ...   
127    48167.0
128    48201.0
129    48439.0
130    51153.0
131    53053.0
Name: longfips1, Length: 90, dtype: float64

In [127]:
51019 in fips_num

True